# Quick look

What are the low hanging fruits?
* cluster members based on Cantat-Gaudin+2018
* youngest clusters
* nearby
* observable with MuSCATs

In [28]:
cc = cr.ClusterCatalog(catalog_name='Babusiaux2018')
df = cc.query_catalog()
df = df.sort_values(by='log10_age')
df['age_Myr'] = df['log10_age'].apply(lambda x: (10**x)/1e6)
#observable with MuSCATs
df[df['dec']>-30].head()

,Cluster,dist_mod,log10_age,[Fe/H],E(B−V),Memb,ra,dec,distance,age_Myr
12,NGC_2232,7.575,7.70,0.11,0.031,241.0,96.684000,-4.732500,327.340695,50.118723
5,alpha_Per,6.214,7.85,0.14,0.090,598.0,51.080709,49.861179,174.904104,70.794578
2,Pleiades,5.667,8.04,-0.01,0.045,1059.0,56.750000,24.116667,135.956508,109.647820
8,Blanco_1,6.876,8.06,0.03,0.010,361.0,1.029167,-29.833333,237.246602,114.815362
20,NGC_2422,8.436,8.11,0.09,0.090,572.0,114.137700,-14.473300,486.631256,128.824955


I'll focus on ngc 2232 because it's young and relatively nearby

In [29]:
df_mem = cc.get_members_Babusiaux2018()
ngc2232 = df_mem[df_mem['Cluster']=='NGC 2232']
ngc2232

,SourceId,Cluster,ra,dec,Gmag
14476,3007192996647388160,NGC 2232,97.29502,-6.62956,14.630
14477,3116923531247562240,NGC 2232,95.74332,-3.34348,17.603
14478,3116924768198184192,NGC 2232,95.92643,-3.30352,18.229
14479,3116966450859538944,NGC 2232,95.79738,-3.06414,17.282
14480,3116962155892150656,NGC 2232,96.09716,-3.01184,17.381
...,...,...,...,...,...
14854,3104155456768550016,NGC 2232,97.49670,-5.36212,18.682
14855,3104173525693960704,NGC 2232,97.24721,-5.22617,16.239
14856,3104159541280761216,NGC 2232,97.28538,-5.46041,18.320
14857,3104179091970711808,NGC 2232,96.76366,-5.54861,19.317


## QL

In [36]:
import os
import matplotlib.pyplot as pl
import numpy as np

import lightkurve as lk
from transitleastsquares import transitleastsquares as tls
import astropy.units as u
import deepdish as dd
import chronos as cr

def make_tql(gaiaid, sector=None, savefig=False, outdir='.', verbose=False):
    l = cr.LongCadence(gaiaDR2id=gaiaid, sector=sector, 
                       sap_mask='square', aper_radius=1,
                       #sap_mask='threshold', threshold_sigma=5,
                       cutout_size=(20,20), verbose=verbose, clobber=False)
    if (outdir is not None) & (not os.path.exists(outdir)):
        os.makedirs(outdir)
        
    fig, axs = pl.subplots(2,3, figsize=(12,8))
    axs = axs.flatten()

    #+++++++++++++++++++++ax0: tpf
    ax = axs[0]
    if verbose:
        print('Querying Tesscut\n')
    tpf = l.get_tpf_tesscut(sector=l.sector)
    gaia_sources = l.query_gaia_dr2_catalog(radius=120)
    _ = cr.plot_gaia_sources_on_tpf(tpf=tpf, 
                                    target_gaiaid=l.gaiaid, 
                                    gaia_sources=gaia_sources,
                                    sap_mask=l.sap_mask, 
                                    aper_radius = l.aper_radius,
                                    #threshold_sigma=l.threshold_sigma,
                                    ax=ax)
#     #+++++++++++++++++++++ax1: raw lc
#     ax = axs[1]
    aper_mask = cr.parse_aperture_mask(tpf, sap_mask=l.sap_mask, 
                                       aper_radius = l.aper_radius,
                                      # threshold_sigma=l.threshold_sigma
                                      )
    raw_lc = tpf.to_lightcurve(aperture_mask=aper_mask)
#     raw_lc.scatter(ax=ax, label='raw')
#     ax.legend()

    #+++++++++++++++++++++ax1: bkg-subtracted lc
    if verbose:
        print('Performing background subtraction\n')
    idx = (
        np.isnan(raw_lc.time)
        | np.isnan(raw_lc.flux)
        | np.isnan(raw_lc.flux_err)
    )
    raw_lc = raw_lc[~idx]
    # Make a design matrix and pass it to a linear regression corrector
    regressors = tpf.flux[~idx][:, ~aper_mask]
    dm = (
        lk.DesignMatrix(regressors, name="pixels")
        .pca(nterms=5)
        .append_constant()
    )

    # Regression Corrector Object
    rc = lk.RegressionCorrector(raw_lc)
    bkg_sub_lc = rc.correct(dm)
#     bkg_sub_lc.normalize().scatter(ax=ax, label='bkg_sub')
#     bkg_sub_lc.normalize().bin(10).scatter(ax=ax, marker='o', label='bkg_sub (bin=10)')

    #+++++++++++++++++++++ax2: Detrending/ Flattening
    ax = axs[1]
    lc = bkg_sub_lc.normalize().remove_nans().remove_outliers()
    flat, trend = lc.flatten(window_length=31, return_trend=True)
    _ = lc.scatter(ax=ax, label='bkg_sub')
    trend.plot(ax=ax, label='trend', lw=3, c='r')
    
    ax = axs[2]
    flat.scatter(ax=ax, label='flat')
    flat.bin(10).scatter(ax=ax, label='flat (bin=10)')
    
    #+++++++++++++++++++++ax3: TLS periodogram
    ax = axs[3]
    lc = flat
    tls_results = tls(lc.time, lc.flux).power()

    label = f"Best period={tls_results.period:.3}"
    ax.axvline(tls_results.period, alpha=0.4, lw=3, label=label)
    ax.set_xlim(np.min(tls_results.periods), np.max(tls_results.periods))

    for i in range(2, 10):
        ax.axvline(i * tls_results.period, alpha=0.4, lw=1, linestyle="dashed")
        ax.axvline(tls_results.period / i, alpha=0.4, lw=1, linestyle="dashed")
    ax.set_ylabel(r"SDE")
    ax.set_xlabel("Period (days)")
    ax.plot(tls_results.periods, tls_results.power, color="black", lw=0.5)
    ax.set_xlim(0, max(tls_results.periods))
    ax.set_title('TLS Periodogram')
    ax.legend()

    #+++++++++++++++++++++ax4: phase-folded
    ax = axs[4]

    ax.plot(tls_results.model_folded_phase, 
               tls_results.model_folded_model, color="red")
    ax.scatter(tls_results.folded_phase, tls_results.folded_y,
        color="blue",
        s=10,
        alpha=0.5,
        zorder=2,
    )
    ax.set_xlabel("Phase")
    ax.set_ylabel("Relative flux")
    ax.set_xlim(0.2,0.8)

    #+++++++++++++++++++++summary
    ax = axs[5]
    tic_params = l.query_tic_catalog(return_nearest_xmatch=True)
    Rp = tls_results['rp_rs']*l.tic_params['rad']*u.Rsun.to(u.Rearth)

    msg="Candidate Properties\n"
    msg+="-"*30+"\n"
    msg+=f"SDE={tls_results.SDE:.2f}\n"
    msg+=f"Period={tls_results.period:.2f}+/-{tls_results.period_uncertainty:.2f} d"+" "*5
    msg+=f"T0={tls_results.T0:.2f} BTJD\n"
    msg+=f"Duration={tls_results.duration:.2f} d\n"
    msg+=f"Depth={tls_results.depth:.4f}\t"
    msg+=f"Rp={Rp:.2f} "+r"R$_{\oplus}$"+"\n"
    msg+=f"Odd-Even mismatch={tls_results.odd_even_mismatch:.2f}\n"

    msg+="\n"*2
    msg+="Stellar Properties\n"
    msg+="-"*30+"\n"
    msg+=f"TIC ID={int(l.tic_params['ID'])}"+" "*5
    msg+=f"Tmag={l.tic_params['Tmag']:.2}\n"
    msg+=f"Rstar={l.tic_params['rad']:.2f}+/-{l.tic_params['e_rad']:.2f} "+r"R$_{\odot}$"+" "*5
    msg+=f"Mstar={l.tic_params['mass']:.2f}+/-{l.tic_params['e_mass']:.2f} "+r"M$_{\odot}$"+"\n"
    msg+=f"Teff={int(l.tic_params['Teff'])}+/-{int(l.tic_params['e_Teff'])} K"+" "*5
    msg+=f"logg={l.tic_params['logg']:.2f}+/-{l.tic_params['e_logg']:.2f} dex\n"
    msg+=r"$\rho$"+f"star={l.tic_params['rho']:.2f}+/-{l.tic_params['e_rho']:.2f} gcc\n"
    msg+=f"Contamination ratio={l.tic_params['contratio']:.2f}\n"
    ax.text(0, 0, msg, fontsize=10)
    ax.axis('off')
    
    ticid = l.tic_params['ID']
    fig.suptitle(f'TIC {ticid} (sector {l.sector})')
    fig.tight_layout()
    
    if savefig:
        fp = os.path.join(outdir, f'{ticid}_s{l.sector}')
        fig.savefig(fp+'.png', bbox_inches='tight')
        dd.io.save(fp+'_tls.h5', tls_results)
        if verbose:
            print(f'Saved: {fp}.png')
    return fig

In [ ]:
from tqdm import tqdm

for gaiaid in tqdm(ngc2232['SourceId'].values):
    try:
        fig = make_tql(gaiaid=gaiaid, savefig=True, outdir='../ngc2232', verbose=False);
    except Exception as e:
        print(e)